# Minigrid - Performance Test

In [1]:
import numpy as np
import gym
import gym_minigrid
import h5py

Read Q-Table from file

In [2]:
hf = h5py.File('qTableLavaV3.h5', 'r')
qtable=hf['qTableLava'][()]
hf.close()

Environments to test

In [3]:
environments=['MiniGrid-LavaCrossingS9N1-v0','MiniGrid-LavaCrossingS9N2-v0','MiniGrid-LavaCrossingS9N3-v0','MiniGrid-LavaCrossingS11N5-v0']

Print the list of the environments name.

In [4]:
for n in environments:
    print(n)

MiniGrid-LavaCrossingS9N1-v0
MiniGrid-LavaCrossingS9N2-v0
MiniGrid-LavaCrossingS9N3-v0
MiniGrid-LavaCrossingS11N5-v0


In [5]:
def get_position(observation,action, position):
    dir=observation['direction']
    x=0;y=0;
    nx=position[0]
    ny=position[1]
    if (action==2): #forward
        if (dir==0):
            y=1
        elif (dir==1):
            x=1
        elif (dir==2):
            y=-1
        else:
            x=-1
        if((nx+x)>=0):
            nx+=x
        if((ny+y)>=0):
            ny+=y
    return [nx,ny]

In [6]:
def mapping(position, map_list):
    nmap=[]
    nx=position[0]
    ny=position[1]
    if not[nx,ny] in map_list:
        map_list.append([nx,ny])
        nmap=map_list
    else:
        nmap=map_list
    return nmap

In [7]:
def pos_visited(position,map_list):
    if position in map_list:
        return True
    else:
        return False

In [8]:
#Regresa el estado observable expresado de acuerdo a los posibles objetos adelante, a la derecha y a la izquierda
def conv_state3(state,vFlag):
    num_obj=11 # 11 posibles objetos por casilla = 11 x 11 x 11 = 1331
    col=[];row=[]
    for c in state:
        col=[]
        for r in c:
            obj=r[0]
            col.append(obj)
        row.append(col)
    front_obj = row[3][5]
    right_obj = row[4][6]
    left_obj = row[2][6]
    state = vFlag * (num_obj**3)+left_obj * (num_obj**2) + front_obj *(num_obj**1) + right_obj *((num_obj**0))
    obj_str=list(gym_minigrid.minigrid.OBJECT_TO_IDX.keys())[front_obj]
    return row,state,obj_str

In [9]:
TEST_PER_ENVIRONMENT=1000
TEST_STEPS = 100
result_actions=[]
result_states=[]
results={}
for e in environments:
    results[e]={'actions':[],'states':[],'performance':0}

for e in environments:
    print("Simulation of ",e," in progress...")
    success=0
    fail=0
    for i in range(TEST_PER_ENVIRONMENT):
        env = gym.make(e)
        cState, state, stateStr = conv_state3(env.reset()['image'],0)
        map_grid=[]
        position=[0,0]
        map_grid.append(position)
        actions=[]
        states=[]
        step_counter=0
        for step in range(TEST_STEPS):
            action=np.argmax(qtable[state,:])
            actions.append(action)

            new_state,reward,done,info=env.step(action)

            position=get_position(new_state,action,position)     # Calcula la posición
            visitedFlag=pos_visited(position,map_grid)

            cState, newState, newStr=conv_state3(new_state['image'],visitedFlag)
            map_grid=mapping(position,map_grid)                  # mapea el recorrido del agente

            states.append(newState)
            state=newState
            
            step_counter+=1
            if done == True:
                results[e]['actions'].append(actions)
                results[e]['states'].append(states)
                if(step_counter<TEST_STEPS):
                    success+=1
                else:
                    fail+=1
                break
    results[e]['performance']=success/TEST_PER_ENVIRONMENT   
print("Simulations Done.\n")
print("\nResults:\n")
for e in environments:
    print(e,"performance: {:.2f} %".format(results[e]['performance']*100))

Simulation of  MiniGrid-LavaCrossingS9N1-v0  in progress...
Simulation of  MiniGrid-LavaCrossingS9N2-v0  in progress...
Simulation of  MiniGrid-LavaCrossingS9N3-v0  in progress...
Simulation of  MiniGrid-LavaCrossingS11N5-v0  in progress...
Simulations Done.


Results:

MiniGrid-LavaCrossingS9N1-v0 performance: 73.90 %
MiniGrid-LavaCrossingS9N2-v0 performance: 43.30 %
MiniGrid-LavaCrossingS9N3-v0 performance: 25.70 %
MiniGrid-LavaCrossingS11N5-v0 performance: 11.10 %
